In [1]:
import ollama

response = ollama.chat(
    model="gemma2:2b",
    # model="qwen2.5:1.5b",
    messages=[{"role": "user", "content": "안녕하세요! 이 모델이 정상적으로 동작하나요?"}]
)

print("🧠 모델 응답:", response['message'])


🧠 모델 응답: role='assistant' content=' 안녕하세요! 저는 Google에서 개발한 대규모 언어 모델입니다. \n\n네, 저는 정상적으로 작동하고 있습니다. 😊  무엇을 도와드릴까요? \n\n\n**(Korean: 안녕하세요! 저는 Google에서 개발한 대규모 언어 모델입니다. 네, 저는 정상적으로 작동하고 있습니다. 😊 무엇을 도와드릴까요?)**\n' images=None tool_calls=None


In [2]:
import json

# ✅ JSON 데이터 로드
import json
import os
from glob import glob

# ✅ info 폴더 내 모든 JSON 파일 로드 함수
def load_data(folder_path="info"):
    data = []
    json_files = glob(os.path.join(folder_path, "*.json"))  # 폴더 내 모든 .json 파일 찾기

    for file_path in json_files:
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                file_data = json.load(f)
                if isinstance(file_data, list):
                    data.extend(file_data)  # 리스트 형태라면 기존 데이터에 추가
                else:
                    print(f"⚠️ {file_path}의 데이터 형식이 리스트가 아님. 무시됨.")
        except Exception as e:
            print(f"❌ JSON 파일 로드 중 오류 발생 ({file_path}): {e}")

    print(f"✅ 총 {len(data)}개의 데이터를 로드했습니다.")
    return data

# ✅ JSON 파일 로드 실행
all_data = load_data("info")

# ✅ 데이터 확인
print(f"📌 총 {len(all_data)} 개의 데이터 로드 완료!")



✅ 총 984개의 데이터를 로드했습니다.
📌 총 984 개의 데이터 로드 완료!


In [3]:

import json
import os
import chromadb
from sentence_transformers import SentenceTransformer

# ✅ ChromaDB 클라이언트 설정
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="document_embeddings")

# ✅ Sentence Transformer 모델 로드
embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ✅ JSON 파일이 없으면 데이터 생성 후 저장
json_file_path = "processed_data.json"
if not os.path.exists(json_file_path):
    print("⚠️ 'processed_data.json' 파일이 존재하지 않습니다. 새로운 데이터 생성 중...")

    # 샘플 데이터 생성
    text_data = [
        {"title": "국가장학금 신청 안내", "content": "국가장학금 신청 기간은 2월 4일부터 3월 18일까지입니다.", "url": "https://www.kosaf.go.kr"},
        {"title": "성적우수 장학금", "content": "직전 학기 평점 4.0 이상 학생들에게 지급됩니다.", "url": "https://www.university.com/scholarship"},
        {"title": "전주대 장학금 정보", "content": "전주대학교에서는 다양한 교내 장학금을 운영 중입니다.", "url": "https://www.jj.ac.kr"},
    ]

    # JSON 파일 저장
    with open(json_file_path, "w", encoding="utf-8") as f:
        json.dump(text_data, f, ensure_ascii=False, indent=4)

    print(f"✅ {json_file_path} 파일을 생성하고 데이터를 저장했습니다.")

# ✅ JSON 데이터 로드
with open(json_file_path, "r", encoding="utf-8") as f:
    text_data = json.load(f)

# ✅ ChromaDB에 데이터 추가
if collection.count() == 0:
    print("⚠️ ChromaDB에 데이터가 없습니다. 새로 삽입합니다...")

    for item in text_data:
        doc_id = item.get("url", item["title"])  # 문서의 URL 또는 제목을 ID로 사용
        collection.add(
            ids=[doc_id],  # 유니크 ID
            documents=[item["content"]],  # ✅ content(문서 본문)만 전달
            metadatas=[{"title": item["title"], "url": item.get("url", "URL 없음")}],  # 추가 메타데이터
            embeddings=[embedding_model.encode(item["content"]).tolist()]  # 문서 내용 임베딩
        )

    print(f"✅ {len(text_data)}개의 데이터를 ChromaDB에 저장 완료!")

# ✅ ChromaDB 검색 함수 (FAISS 제거 완료, 키워드 가중치 제거)
def search_data(query, top_k=5):
    query_embedding = embedding_model.encode(query).tolist()  # 질문 벡터화

    # ChromaDB에서 유사한 문서 검색
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    retrieved_data = []
    if results["documents"]:  # 검색 결과가 존재하는 경우
        for i, doc_content in enumerate(results["documents"][0]):  # 🔥 doc_content는 단순 문자열
            retrieved_data.append({
                "title": results["metadatas"][0][i]["title"],  # 🔥 title을 metadatas에서 가져옴
                "content": doc_content,  # 🔥 content는 ChromaDB에서 반환된 문자열
                "url": results["metadatas"][0][i].get("url", "URL 없음"),  # 🔥 url도 metadatas에서 가져옴
                "score": results["distances"][0][i]  # ChromaDB에서는 거리 기반 정렬
            })

    if not retrieved_data:
        print("⚠️ 관련된 문서를 찾지 못했습니다.")
        return []

    # ✅ 점수가 낮은(우선순위 높은) 순서로 정렬 (키워드 가중치 제거)
    retrieved_data = sorted(retrieved_data, key=lambda x: x["score"])

    return retrieved_data

print("✅ ChromaDB 기반 검색 시스템 적용 완료! 키워드 가중치 제거 완료!")

# ✅ 실행 테스트
query = "국가장학금 신청 방법"
answer = search_data(query)

# 결과 출력
for res in answer:
    print(f"📌 {res['title']}: {res['content'][:300]}... (Score: {res['score']})")


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


✅ ChromaDB 기반 검색 시스템 적용 완료! 키워드 가중치 제거 완료!
📌 전주대 장학금 정보: 전주대학교에서는 다양한 교내 장학금을 운영 중입니다.... (Score: 11.251113963947407)
📌 국가장학금 신청 안내: 국가장학금 신청 기간은 2월 4일부터 3월 18일까지입니다.... (Score: 11.794244487222851)
📌 성적우수 장학금: 직전 학기 평점 4.0 이상 학생들에게 지급됩니다.... (Score: 13.803185207375408)


### 🔄️ Allora

In [ ]:
import json
import gradio as gr
import ollama
import torch
import re
from sentence_transformers import SentenceTransformer, util
from langdetect import detect
from deep_translator import GoogleTranslator

# ✅ 문장 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# ✅ 정확한 언어 감지를 위한 개선된 함수
def detect_language(text):
    """ langdetect를 사용하여 언어 감지 """
    try:
        lang = detect(text)
        return lang  # 'ko' 또는 'en' 반환
    except:
        return "ko"  # 기본값은 한국어

# ✅ 텍스트 번역 함수 (추후 확장 가능)
def translate_text(text, target_language="en"):
    """ 번역 API 연결 (추후 확장 가능) """
    return text  # 현재는 그대로 반환하지만, 실제 번역 API 연결 가능

# ✅ 각 학교명에 해당하는 JSON 파일 매핑
school_json_map = {
    "영진전문대": "./info/영진전문대_최종.json",
    "Yeungjin College": "./info/영진전문대_최종.json",
    "YJU": "./info/영진전문대_최종.json",
    "경북대": "./info/경북대_최종.json",
    "Kyungpook University": "./info/경북대_최종.json",
    "KNU": "./info/경북대_최종.json",
    "전주대": "./info/전주대_최종.json",
    "Jeonju University": "./info/전주대_최종.json",
    "JJU": "./info/전주대_최종.json",
    "서울시립대": "./info/시립대_최종.json",
    "UOS": "./info/시립대_최종.json",
    "서울과학기술대": "./info/과기대_최종.json",
    "SNUST": "./info/과기대_최종.json",
    "전남대": "./info/전남대_최종.json",
    "Chonnam University": "./info/전남대_최종.json",
    "JNU": "./info/전남대_최종.json",
    "CNU": "./info/전남대_최종.json",
    "성균관대": "./info/성균관대_최종.json",
    "성대": "./info/성균관대_최종.json",
    "성균관": "./info/성균관대_최종.json",
    "Sungkyunkwan University": "./info/성균관대_최종.json",
    "SKKU": "./info/성균관대_최종.json",
}

def detect_school(query):
    """ 사용자의 질문에서 학교명을 감지 """
    for school in school_json_map.keys():
        if school in query:
            return school
    return None  # 해당하는 학교 없음

def load_data(school):
    """ 특정 학교의 JSON 데이터만 로드 """
    json_file = school_json_map.get(school)
    if not json_file:
        return []
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"⚠️ 파일 로드 오류: {json_file}, {e}")
        return []

def search_data(query, school):
    """ 문장 의미 기반 검색 (Sentence Embedding) """
    all_data = load_data(school)
    if not all_data:
        return []

    query_embedding = embedding_model.encode(query, convert_to_tensor=True)

    scores = []
    for item in all_data:
        doc_text = f"{item['title']} {item['content']}"
        doc_embedding = embedding_model.encode(doc_text, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, doc_embedding).item()

        if similarity > 0.5:  # 유사도가 50% 이상일 때만 포함
            scores.append((similarity, item))

    scores.sort(reverse=True, key=lambda x: x[0])

    return [res[1] for res in scores[:5]]  # 최대 5개 결과 반환

def clean_response(response_text):
    """ 응답에서 불필요한 부분을 제거하는 함수 """

    # ✅ "role='assistant' content=" 제거
    response_text = re.sub(r"role=['\"]assistant['\"] content=['\"]", "", response_text)

    # ✅ "\n" (이스케이프 문자), "\r" 제거
    # response_text = response_text.replace("\\n", "     ").replace("\r", " ")

    # ✅ 유니코드 특수 문자 제거 (예: \u200d)
    response_text = re.sub(r"\u200d", "", response_text)

    # ✅ "**영어:**", "**Korean:**" 제거
    response_text = re.sub(r"\*\*영어:\*\*|\*\*Korean:\*\*", "", response_text)

    # ✅ "images=None tool_calls=None" 제거
    response_text = re.sub(r"images=None tool_calls=None", "", response_text)

    # ✅ 여러 개의 공백을 단일 공백으로 정리
    response_text = re.sub(r"\s+", " ", response_text)

    # ✅ 앞뒤 공백 정리
    response_text = response_text.strip()

    return response_text

def generate_response(query):
    """ 사용자의 질문을 분석하고, 해당 학교 JSON 데이터에서 응답 생성 """
    detected_lang = detect_language(query)  # ✅ 개선된 언어 감지 적용

    school = detect_school(query)

    if not school:
        return "⚠️ 학교명을 정확하게 입력해주세요. 예) '시립대 성적 장학금 알려줘'"

    search_results = search_data(query, school)

    if not search_results:
        return f"⚠️ {school} 관련 정보를 찾을 수 없습니다. 다른 키워드로 검색해보세요!"

    seen_urls = set()
    filtered_context = []
    for res in search_results:
        url = res.get('url', 'URL 없음')
        if url not in seen_urls:
            filtered_context.append(f"📌 {res['title']} (URL: {url})\n  {res['content'][:200]}...")
            seen_urls.add(url)

    context = "\n".join(filtered_context)

    prompt = f"""
    사용자의 질문: "{query}"

    - 아래는 {school} 관련 검색된 정보입니다. 제공된 정보 외에 추가적인 내용을 생성하지 마세요.
    - 대답에는 해당 정보의 url을 링크형식으로 반드시 한번씩은 꼭 제공해주세요!
    - 그리고 문단, 문장, 답변을 정리할때 이모티콘을 최대한 많이 활용해주세요.
    - **반드시 질문과 동일한 언어로 답변하세요!** 
    - **예외 없이 질문이 영어면 영어로, 한국어면 한국어로만 답변해야 합니다.**

    답변에는 json 파일에 있는 정보를 최대한 활용해주세요. content 부분의 내용을 요약하여 답변에 포함해주세요.
    검색된 정보:
    {context}

    위 정보를 반드시 참고,분석하여 최대한 정확하고 자연스러운 답변을 생성하세요.
    """

    response = ollama.chat(
        model="gemma2:2b",
        # model="qwen2.5:1.5b",
        messages=[{"role": "user", "content": prompt}]
    )

    chatbot_response = str(response['message'])

    # ✅ 불필요한 텍스트 정리
    chatbot_response = clean_response(chatbot_response)
    response_lang = detect_language(chatbot_response)
    if response_lang != detected_lang:
        chatbot_response = translate_text(chatbot_response, target_language=detected_lang)

    return chatbot_response

# ✅ Gradio UI 생성
def chat_interface(user_input):
    return generate_response(user_input)

# ✅ Gradio 앱 실행행
# with gr.Blocks() as app:
#     gr.Markdown("# 🎓 대학 장학금 정보 챗봇 💬")
#     gr.Markdown("### 원하는 대학교와 장학금 관련 질문을 입력하세요!")

#     user_input = gr.Textbox(label="질문을 입력하세요 (예: 성균관대 성적 장학금)")
#     output = gr.Textbox(label="챗봇의 답변")

#     submit_button = gr.Button("검색하기")

#     submit_button.click(fn=chat_interface, inputs=user_input, outputs=output)

# app.launch(share=True)

import gradio as gr

def chat_interface(user_input):
    response = generate_response(user_input)
    response = response.replace("\\n", "<br>")  # ✅ HTML 줄바꿈 태그 적용
    response = response.replace("####", "💠")  # ✅ HTML 줄바꿈 태그 적용
    response = response.replace("###", "🔹")  # ✅ HTML 줄바꿈 태그 적용
    response = response.replace("\\u200d", " ")  # ✅ HTML 줄바꿈 태그 적용
    return response

with gr.Blocks() as app:
    gr.Markdown("# 🎓 대학 장학금 정보 챗봇 💬")
    gr.Markdown("### 원하는 대학교와 장학금 관련 질문을 입력하세요!")

    user_input = gr.Textbox(label="질문을 입력하세요 (예: 성균관대 성적 장학금)")
    output = gr.Markdown()  # ✅ Markdown을 사용하여 HTML 렌더링 가능하게 설정

    submit_button = gr.Button("검색하기")

    submit_button.click(fn=chat_interface, inputs=user_input, outputs=output)

app.launch(share=True)




* Running on local URL:  http://127.0.0.1:7873

Could not create share link. Missing file: C:\Users\juneo\AppData\Roaming\Python\Python312\site-packages\gradio\frpc_windows_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_arm64.exe
2. Rename the downloaded file to: frpc_windows_arm64_v0.3
3. Move the file to this location: C:\Users\juneo\AppData\Roaming\Python\Python312\site-packages\gradio
